In [1]:
%run ../yp_utils.py

# Initial setup

In [2]:
paper_pmid = 28520786
paper_name = 'choi_oshea_2017' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [12]:
xl = pd.ExcelFile('raw_data/journal.pone.0176085.s004.xlsx')
sheet_names = np.array(xl.sheet_names)[[0,2]]

In [13]:
sheet_names

array(['all single mutants in 50 µM Pi', 'all single mutants in 1 mM Pi'],
      dtype='<U30')

In [18]:
original_data_list = []
for s in sheet_names:
    original_data = xl.parse(s)
    print('Original data dimensions: %d x %d' % (original_data.shape))
    original_data['orf'] = original_data['Gene_systematic name'].astype(str)
    original_data['orf'] = clean_orf(original_data['orf'])
    original_data.loc[original_data['orf']=='YLR287-A','orf'] = 'YLR287C-A'
    original_data['orf'] = translate_sc(original_data['orf'], to='orf')
    t = looks_like_orf(original_data['orf'])
    print(original_data.loc[~t,])
    original_data['data'] = original_data.iloc[:,2].astype(float)
    original_data.set_index('orf', inplace=True)
    original_data = original_data[['data']].copy()
    original_data = original_data.groupby(original_data.index).mean()
    print(original_data.shape)
    
    original_data_list.append(original_data)

Original data dimensions: 5344 x 6
Empty DataFrame
Columns: [Gene_systematic name, Gene_generic name, average_log2(YFP/RFP) (0=wild type in 50 µM Pi), standard deviation_log2(YFP/RFP), p-value of measurement errors, detected number, orf]
Index: []
(5316, 1)
Original data dimensions: 5332 x 6
Empty DataFrame
Columns: [Gene_systematic name, Gene_generic name, average_log2(YFP/RFP) (0=wild type in 1mM Pi), standard deviation_log2(YFP/RFP), p-value of measurement errors, detected number, orf]
Index: []
(5303, 1)


In [19]:
original_data1, original_data2 = original_data_list

In [20]:
original_data = original_data1.join(original_data2, how='outer', lsuffix='_1', rsuffix='_2')

In [25]:
# M&M indicates that the list includes 4974 knockout alleles of
# nonessential genes and 878 hypomorphic alleles of essential genes. Must
# eliminate the essential genes (no other way available)

noness = ~is_essential(original_data.index.values)
original_data = original_data.loc[noness.values,:]
print(original_data.shape)

Loading SGD features from 2017-04-03.
(4575, 2)


In [26]:
dataset_ids = [11807, 11808]

# Prepare the final dataset

In [27]:
data = original_data.copy()

In [28]:
datasets = datasets.reindex(index=dataset_ids)

In [29]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [30]:
data.head()

dataset_id,11807,11808
data_type,value,value
orf,,
YAL002W,1.400102,-0.493898
YAL004W,-0.567216,0.110620
YAL005C,-0.484751,0.138782
YAL007C,1.169149,-0.011764
YAL008W,0.140533,0.016276


## Subset to the genes currently in SGD

In [31]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 23


In [32]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

In [33]:
data.head()

,dataset_id,11807,11808
,data_type,value,value
gene_id,orf,,
2,YAL002W,1.400102,-0.493898
1863,YAL004W,-0.567216,0.110620
4,YAL005C,-0.484751,0.138782
5,YAL007C,1.169149,-0.011764
6,YAL008W,0.140533,0.016276


# Normalize

In [34]:
data_norm = normalize_phenotypic_scores(data, has_tested=True)

In [35]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [36]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

In [37]:
data_all.head()


,dataset_id,11807,11808,11807,11808
,data_type,value,value,valuez,valuez
gene_id,orf,,,,
2,YAL002W,1.400102,-0.493898,1.961961,-1.467685
1863,YAL004W,-0.567216,0.110620,-0.721889,0.401048
4,YAL005C,-0.484751,0.138782,-0.609388,0.488105
5,YAL007C,1.169149,-0.011764,1.646890,0.022723
6,YAL008W,0.140533,0.016276,0.243635,0.109405


# Print out

In [38]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [39]:
from IO.save_data_to_db3 import *

In [40]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/2 [00:00<?, ?it/s]

Deleting all datasets for PMID 28520786...
Inserting the new data...


100%|██████████| 2/2 [00:13<00:00,  6.68s/it]

Updating the data_modified_on field...
